In [13]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

ModuleNotFoundError: No module named 'cv2'

In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import cv2

# Define the directory where the images are stored
def load_images_from_folder(base_dir, subfolders):
# Iterate over all subfolders and load images
    images = []
    labels = []
    for subfolder in subfolders:
        folder_path = os.path.join(base_dir, subfolder)
        print(f"Loading images from: {folder_path}")
        label = subfolder
    
        for filename in os.listdir(folder_path):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(folder_path, filename)
                img = Image.open(os.path.join(img_path, filename))
                try:
                    images.append(img)
                    labels.append(label)
                    print(f"images loaded {filename}")
                except Exception as e:
                    print(f"Error loading image {filename}: {e}")
    return images, labels

X, y = load_images_from_folder(
#base_dir = r"C:\Users\bejdi\Desktop\Micro-Doppler-Based-Target-Classification-\data\DIAT-uSAT_dataset",

# List of all subfolders
subfolders = [r"3_long_blade_rotor\3_long_blades_rotor", r"3_short_blade_rotor_1\3_short_blade_rotor_1", r"Bird\Bird", 
              r"Bird+mini-helicopter_1\Bird+2_Blade_rotor_1", r"drone_1\drone_1", r"drone_2\drone_2", r"RC plane_1\2_blade_rotor_1", r"RC plane_2\2_blade_rotor_2"])

In [ ]:
X = np.array(X)
print(X)

In [ ]:
def convert_to_numpy_and_resize(X, target_size=(224, 224)):
    """
    Converts a list of images (PIL or other image objects) to a NumPy array 
    and resizes them to the target size.

    Parameters:
    X (list): A list containing loaded images.
    target_size (tuple): The target size for resizing the images (width, height).

    Returns:
    np.ndarray: A NumPy array of resized images.
    """
    resized_images = []
    for img in X:
        # If the image is not already a NumPy array, convert it
        if not isinstance(img, np.ndarray):
            img = np.array(img)
        # Resize the image using TensorFlow (ensure the image has 3 channels)
        img_resized = tf.image.resize(img, target_size)
        img_resized = np.array(img_resized)  # Convert back to NumPy array if needed
        resized_images.append(img_resized)
    
    return np.array(resized_images)

X_resized = convert_to_numpy_and_resize(X)
y = np.array(y)  # Ensure labels are also a NumPy array

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_resized, y, test_size=0.2, random_state=42)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

#size of dataset
print(f"Training set size: {X_train.shape}, Training labels size: {y_train.shape}")
print(f"Validation set size: {X_val.shape}, Validation labels size: {y_val.shape}")
print(f"Testing set size: {X_test.shape}, Testing labels size: {y_test.shape}")

In [ ]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)
y_val_encoded = label_encoder.transform(y_val)

In [ ]:
num_classes = len(np.unique(y))
y_train_cat = to_categorical(y_train_encoded, num_classes)
y_test_cat = to_categorical(y_test_encoded, num_classes)
y_val_cat = to_categorical(y_val_encoded, num_classes)